In [1]:
#Importing required packages
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt

# Activity-1:Retrieving the K-mers from the sequence

In [2]:
#Function used to retrieve the kmers
def get_kmers(seq,k):
    kmerDict={}
    for i in range(len(seq)-k+1):
        kmerDict[i] = seq[i:i+k]
    return kmerDict

In [3]:
#Kmer length
k = 3
#Given sequences
seq1 = "TAATGCC"
seq2 = "GCGGTAATGCAGTTGAC"
seq3 = "TTGAGTGCCAGCG"

In [4]:
#Obtaining the kmer sequences
K1 = get_kmers(seq1,k)
K1

{0: 'TAA', 1: 'AAT', 2: 'ATG', 3: 'TGC', 4: 'GCC'}

# Activity-2:Lexicographic Ordering of K-mers

In [5]:
#Function used to sort the kmer list in lexicographic order
def kmer_list(dict_K):
    s = [];
    for key,val in dict_K.items():
        #Appending all kmers in the list
        s.append(val)
    #Returning the kmer in list
    return s

In [6]:
#Obtaining the lexicographic order of the kmers
s = kmer_list(K1)
#Sorting done with inbuilt sort command which is based on unicode/ASCII values
s.sort()
print(s)

['AAT', 'ATG', 'GCC', 'TAA', 'TGC']


# Activity-3:Hamiltonian Path Reconstruction & Activity-4:Building the Graph

In [7]:
#Defining the suffix and prefix functions
def suffix(str1):
    return(str1[1:k])
def prefix(str2):
    return(str2[0:k-1])

In [8]:
#Function to create adjacency matrix (a directed graph)
def create_AdjMat(kmer):
    #Creating zeros matrix
    Adj = np.zeros((len(kmer),len(kmer)))
    for i in range(len(kmer)):
        for j in range(len(kmer)):
            #Mapping done based on directed graphs
            if(kmer[i] != kmer[j] and prefix(kmer[j]) == suffix(kmer[i])):
                Adj[i][j] = 1
    return Adj;

In [9]:
#Function to find the starting and ending of the string
def start_end(dictK):
    start = ""
    ending = ""
    #Creating the adjacency matrix
    #print(dictK)
    a2 = create_AdjMat(dictK)
    #print(a2)
    #Computing outdegree from adjacency matrix
    outdegree = a2.sum(axis = 1)
    #Computing indegree from adjacency matrix
    indegree = a2.sum(axis = 0)
    #In order to obtain the starting and ending points accurately and stop the testing once
    #the starting and ending points are known
    count1 = 0
    count2 = 0
    for i in range(len(outdegree)):
        #Obtaining the ending point as it's outdegree is minimum, i.e, 0(ideal)
        if(outdegree[i] == min(outdegree) and count1 < 1):
            ending = ending + str(dictK[i])
            count1 = count1 + 1
        #Obtaining the starting point as it's indegree is minimum, i.e, 0(ideal) 
        elif(indegree[i] == min(indegree) and count2 < 1):
            start = start + str(dictK[i])
            count2 = count2 + 1
    return start,ending,a2
start, ending, adjMat = start_end(s)

In [10]:
#Initialise Graph
Graph = dict();
#Function used to add the edges
def addEdge(edge1, edge2):
    #If an edge1 is already present, edge2 gets appended to the list
    if(edge1 in Graph):
        Graph[edge1].append(edge2);
    else:
        #edge2 assigned at key=edge1 which means edge directed from edge1 to edge2
        Graph[edge1] = [edge2];

In [11]:
#Defining the vertices of the Graph
Vertex = list(set(s));
#print(Vertex)

In [12]:
#Graph is stored in the form of a dictionary
#Creating Graph by passing the edges
for i in range(len(s)):
    for j in range(len(s)):
        if(i != j and suffix(s[i]) == prefix(s[j])):
            addEdge(s[i],s[j])
#Graph

In [13]:
def PathPresent(starting, ending): 
    # Mark all the vertices as not visited
    visited = np.zeros(len(Vertex))
    # Hamiltonian Graph linking
    Ham_graph = "(" + starting + ")" + "--->"
    # Create a queue for Breadth First Search 
    queue = []
    # Enqueue the Starting Vertex(starting)
    queue.append(starting);
    path = str(starting);
    index = 0
    #Find index at which starting value occurs
    for i in range(len(Vertex)):
        if(Vertex[i] == starting):
            index = i;
    #Assigning starting vertex(starting) as visited
    visited[index] = 1;
    path = str(starting);
    while len(queue)>0:
        #Dequeue a vertex from queue  
        inter = queue.pop()
        #If this adjacent node is the destination node, then return true 
        if (inter == ending):
            #print(visited)
            Ham_graph = Ham_graph[0:-4]
            print(Ham_graph)
            return path
        # Else, continue to do Breadth First Search 
        for key,val in Graph.items():
            #Used to find the index at which next node is present in order to mark visited
            index = 0
            #If there is only 1 edge2 from edge1 
            if(len(val) == 1): 
                for idx in range(len(Vertex)):
                    if(val[0] == Vertex[idx]):
                        index = idx
                        break
                if(key == inter and visited[index] == 0):
                    #We queue in the next vertex and mark that location as visited
                    queue.append(val[0])
                    visited[index] = 1
                    path = path + str(val[0][-1])
                    Ham_graph = Ham_graph + "(" + val[0] + ")" + "--->"
            #If there are multiple edge2 from edge1    
            else:
                for i in range(len(val)):
                    index = 0
                    for idx in range(len(Vertex)):
                        if(val[i] == Vertex[idx]):
                            index = idx
                            break
                    if (key == inter and visited[index] == 0):
                         #We queue in the next vertex and mark that location as visited
                        queue.append(Vertex[index]) 
                        visited[index] = 1
                        path = path + str(Vertex[index][-1])
                        Ham_graph = Ham_graph + "(" + Vertex[index] + ")" + "--->"
    # If Breadth First Search is completed without reaching ending Vertex(d) 
    return [] 

In [14]:
#Defining the Starting kmer
vert_start = start;
#Defining the Ending kmer
vert_end = ending;
#Obtaining the reconstructed string
reconstructed_string = PathPresent(vert_start, vert_end)
#Checking whether the path exists or not
if(len(reconstructed_string) > 0):
    print("Path Exist:" + reconstructed_string)
else:
    print("Path Does Not Exist")

(TAA)--->(AAT)--->(ATG)--->(TGC)--->(GCC)
Path Exist:TAATGCC


In [15]:
#Checking whether reconstructed string is same as the initial sequence
reconstructed_string == seq1
#reconstructed_string == seq2
#reconstructed_string == seq3

True